In [ ]:
import numpy as np # support for multi-dimensional arrays and matrices
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

In [ ]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X_test = pd.read_csv('X_test.csv')

In [ ]:
# new feature, 0 if cold (<300 Kelvin), 1 if warm

def is_warm(features):
    warm = []
    for observation in features['reanalysis_avg_temp_k']:
        if observation < 300:
            warm.append(0)
        else:
            warm.append(1)
    return warm

warmth = is_warm(X)
warmth_test = is_warm(X_test)

X['warmth'] = warmth
X_test['warmth'] = warmth_test

In [ ]:
# remove constant columns (std = 0)
remove = []
for col in X.columns:
    if X[col].std() == 0:
        remove.append(col)

X.drop(remove, axis=1, inplace=True)
X_test.drop(remove, axis=1, inplace=True)


print(X.shape, X_test.shape)

#### Inspecting what a Boosting model selects as features

In [ ]:
Cols = X.columns.values.tolist()
clf = GradientBoostingRegressor(random_state = 8001)

selector = clf.fit(X, y)
importances = selector.feature_importances_
fs = SelectFromModel(selector, prefit=True)
X = fs.transform(X)
X_test = fs.transform(X_test)
print(train.shape, test.shape)

In [ ]:
selectedCols = X.shape[1]
sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
sortedCols = sortedCols[0:selectedCols]
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)
X.columns = sortedCols
X_test.columns = sortedCols

print(sortedCols[0:10])

In [ ]:
X = X.replace(np.inf, 999999)
X = X.replace(-np.inf, -999999)
X = X.replace(np.nan, -1)
X_test = X_test.replace(np.inf, 999999)
X_test = X_test.replace(-np.inf, -999999)
X_test = X_test.replace(np.nan, -1)

In [ ]:
# Second round of gradient boosting
Cols = X.columns.values.tolist()
clf = GradientBoostingRegressor(random_state=1729)
selector = clf.fit(X, y)

importances = selector.feature_importances_
fs = SelectFromModel(selector, prefit=True)
X = fs.transform(X)
X_test = fs.transform(X_test)
print(X.shape, X_test.shape)

selectedCols = X.shape[1]
sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
sortedCols = sortedCols[0:selectedCols]

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV

### Grid Search 1

In [ ]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 100, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1) 

In [ ]:
optimized_GBM.fit(X, y)

In [ ]:
optimized_GBM.grid_scores_

### Grid Search 2

In [ ]:
cv_params = {'max_depth': [6,7,8,9], 'min_child_weight': [3,5,7]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 100, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1) 

In [ ]:
optimized_GBM.fit(X, y)

In [ ]:
optimized_GBM.grid_scores_

We pick max_depth: 7 and min_child_weight: 5
Next we vary n_estimators, subsample, and colsample_bytree

### Grid Search 3

In [ ]:
cv_params = {'n_estimators': [75,100,200,300], 'subsample': [0.7,0.8,0.9], 'colsample_bytree': [0.7,0.8,0.9]}
ind_params = {'learning_rate': 0.1, 'min_child_weight': 5, 'seed':0, 'max_depth': 7}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1) 

In [ ]:
optimized_GBM.fit(X, y)

In [ ]:
optimized_GBM.grid_scores_

#### Tuned Model

In [ ]:
xgdmat = xgb.DMatrix(X, y)

In [ ]:
params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
            'max_depth':7, 'min_child_weight':5, 'n_estimators': 300} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['rmse'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

In [ ]:
cv_xgb.tail(5)

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'max_depth':7, 'min_child_weight':5} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 300)

### Tuned Model Predictions

In [ ]:
testdmat = xgb.DMatrix(X_test)

In [ ]:
y_pred = final_gb.predict(testdmat)

In [ ]:
for pred in y_pred:
    print(int(round(pred)))

### Initial Model (cross validation, no tuning grid)

In [ ]:
# Create an empty array for prediction
predictedResult = np.zeros(X.shape[0])

# Split dataset into k = 10 consecutive folds
# Each fold is used once as a validation while the k - 1 remaining folds form the training set
kf = KFold(X.shape[0], n_folds=5)

testPred = []

for trainIndex, testIndex in kf:
    trainFold, testFold = X[trainIndex], X[testIndex]
    trainFoldTarget, testFoldTarget = y[trainIndex], y[testIndex]
    
    xgbc = xgb.XGBRegressor(n_estimators = 300, # number of boosted trees
                             learning_rate = 0.1, # step size shrinkage used in update to prevent overfitting
                             max_depth = 7, # maximum depth of a tree
                             subsample = 0.8, # subsample ratio of the training set (Stochastic gradient boosting)
                             colsample_bytree = 0.8,
                           min_child_weight = 5) # subsample features
    
    xgbc.fit(trainFold, trainFoldTarget)
    xgbpred =xgbc.predict(testFold)

    testPred.append(xgbc.predict(X_test))
    predictedResult[testIndex] = xgbpred
    
    # Print the MA
    print(mean_absolute_error(testFoldTarget, xgbpred))

In [ ]:
print(mean_absolute_error(y, predictedResult))
testPred = np.average(np.array(testPred), axis =0)
#pd.DataFrame({"ID": test_id, "TARGET": testPred}).to_csv('submission.csv',index=False)

In [ ]:
for pred in testPred:
    print(int(round(pred)))